In [ ]:
# This cell can be used for testing or pip install. To remove at final version

%pip install unidecode azure-storage-blob sqlalchemy psycopg2-binary


  Obtaining dependency information for unidecode from https://files.pythonhosted.org/packages/84/b7/6ec57841fb67c98f52fc8e4a2d96df60059637cba077edc569a302a8ffc7/Unidecode-1.3.8-py3-none-any.whl.metadata
  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/57/4f/e1db9475f940f1c54c365ed02d4f6390f884fc95a6a4022ece7725956664/SQLAlchemy-2.0.37-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/5d/f1/09f45ac25e704ac954862581f9f9ae21303cc5ded3d0b775532b407f0e90/psycopg2_binary-2.9.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for greenlet!=0.4.17 from https://files.pythonhosted.org/packages/f7/4b/1c9695aa24f808e156c8f4813f685d975ca73c000c2a5056c514c64980f6/greenlet-3.1.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Keep cell empty to use as start-run cell

In [1]:
# Run this cell to import libraries, connect to Azure postgreSQL flexible server & blob storage

import pandas as pd
import numpy as np
from decimal import Decimal
from unidecode import unidecode
from sqlalchemy import create_engine
from azure.storage.blob import BlobServiceClient, BlobClient
from io import StringIO

# Database connection string 
db_username = 'azurepg'  # Replace with your PostgreSQL username
db_password = 'postgres#1'  # Replace with your PostgreSQL password
db_host = 'azurepg.postgres.database.azure.com'        # Replace with your host if not local
db_port = 5432               # Default PostgreSQL port
db_name = 'postgres'    # Replace with your database name

# Create connection to Azure PostgreSQL server
conn_str = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(conn_str)

# Azure Blob Storage connection details
account_name = "olistblobs"
account_key = "eZaLN3sA0Ze1VEXPaeri6nUQLfSGGOk7guVZ0JIy26XYBhh+aEq/1Bdlpxi4SI+7XXIEe+KP97eX+AStKfyeCQ=="
container_name = "olist"
folder = "olist/"
cleaned_folder = "cleaned/"

# Create a connection to Azure blob storage and container   
container_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key).get_container_client(container_name) 


1. Data Cleaning for "Customers"
- removed duplicates
- strip leading/trailing whitespace
- handling null values
- convert columns to appropriate data types
- ensuring same character length

Issues
- should duplicated 'customer_unique_id' be removed?
- to convert column data type to 'category', the null values have to filled with 'N/A' first before column data type conversion

In [3]:
# Load the dataset
file_name = 'olist_customers_dataset.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))

# Clean the data
def clean_data(df):
    # Step 1: Strip leading/trailing whitespace from all string columns
    str_columns = df.select_dtypes(include='string').columns
    df[str_columns] = df[str_columns].apply(lambda col: col.str.strip())

    # Step 2: Increase readibility of the categorical data
    df.loc[:,'customer_city'] = df['customer_city'].str.title()
    
    # Step 3: Replace NULL values with the string "N/A":
    df = df.fillna('N/A')
    
    # Step 4: Convert columns to appropriate data types
    df['customer_id'] = df['customer_id'].astype('string')
    df['customer_unique_id'] = df['customer_unique_id'].astype('string')
    df['customer_zip_code_prefix'] = df['customer_zip_code_prefix'].astype('string')
    df['customer_city'] = df['customer_city'].astype('category') 
    df['customer_state'] = df['customer_state'].astype('category')
    
    # Step 5: Normalize `customer_zip_code_prefix` to ensure all are 5 characters
    df['customer_zip_code_prefix'] = df['customer_zip_code_prefix'].str.zfill(5)

    # Step 6: Drop customer id column  based on the specified subset of columns
    df = df.drop(columns="customer_id")

    # Step 7: Drop duplicates based on the specified subset of columns
    df = df.drop_duplicates(subset='customer_unique_id')

    # Return the cleaned dataframe
    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_customers_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'customers'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")

                     customer_unique_id customer_zip_code_prefix  \
0      861eff4711a542e4b93843c6dd7febb0                    14409   
1      290c77bc529b7ac935b93aa66c333dc3                    09790   
2      060e732b5b29e8181a18229c7b0b2b5e                    01151   
3      259dac757896d24d7702b9acbbff3f3c                    08775   
4      345ecd01c38d18a9036ed96c73b8d066                    13056   
...                                 ...                      ...   
99436  1a29b476fee25c95fbafc67c5ac95cf8                    03937   
99437  d52a67c98be1cf6a5c84435bd38d095d                    06764   
99438  e9f50caf99f032f0bf3c55141f019d99                    60115   
99439  73c2643a0a458b49f58cea58833b192e                    92120   
99440  84732c5050c01db9b23e19ba39899398                    06703   

               customer_city customer_state  
0                     Franca             SP  
1      Sao Bernardo Do Campo             SP  
2                  Sao Paulo             SP  

2. Data Cleaning for "geolocation"
- removed duplicates, keeping the first occurence
- strip leading/trailing whitespace
- handling null values
- convert columns to appropriate data types

Issues
- should category be used?

In [ ]:
# Load the dataset
file_name = 'olist_geolocation_dataset.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))

# Clean the data
def clean_data(df):
    # Step 1: Remove duplicates in `geolocation_zip_code_prefix`, keeping the first occurrence
    df['geolocation_city'] = df['geolocation_city'].apply(lambda x: unidecode(x))
    df = df.drop_duplicates(subset='geolocation_zip_code_prefix')
    
    # Step 2: Strip leading/trailing whitespace from all string columns
    str_columns = df.select_dtypes(include='string').columns
    df[str_columns] = df[str_columns].apply(lambda col: col.str.strip())

    # Step 3: Increase readibility of the categorical data
    df.loc[:, 'geolocation_city'] = df['geolocation_city'].str.title()

    # Step 4: Replace NULL values with the string "N/A":
    df = df.fillna('N/A')
    
    # Step 5: Convert columns to appropriate data types
    df['geolocation_zip_code_prefix'] = df['geolocation_zip_code_prefix'].astype('string')
    df['geolocation_lat'] = df['geolocation_lat'].astype('float64')
    df['geolocation_lng'] = df['geolocation_lng'].astype('float64')
    df['geolocation_city'] = df['geolocation_city'].astype('category')
    df['geolocation_state'] = df['geolocation_state'].astype('category')

    # Step 6: Normalize `geolocation_zip_code_prefix` to ensure all are 5 characters
    df['geolocation_zip_code_prefix'] = df['geolocation_zip_code_prefix'].str.zfill(5)
    
    # Return the cleaned dataframe
    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_geolocation_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'geolocation'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")

#----------------------------------#
# Extract city names to City table

# Step 1: Extract distinct city_name and state_code. cleaned_data here is referring to geolocation
city = cleaned_data[['geolocation_city', 'geolocation_state']].drop_duplicates().rename(columns={
    'geolocation_city': 'city_name',
    'geolocation_state': 'state_code'
}).reset_index(drop=True)

# Step 2: insert auto-increment city_id
city.insert(0, 'city_id', range(1, len(city) + 1))

# Step 3: Join geolocation and city DataFrames on the city column
merged_df = pd.merge(cleaned_data, city, left_on=['geolocation_city','geolocation_state'], right_on=['city_name','state_code']).rename(columns={
    'geolocation_lat': 'latitude',
    'geolocation_lng': 'longitude'
})

# Step 4: Drop zipcode and duplicated columns
merged_df.drop(columns=['geolocation_zip_code_prefix','geolocation_city','geolocation_state'], inplace=True)

# Step 5: Compute the average latitude and longitude, grouped by city_id
get_lat_lng = merged_df.groupby(['city_id'], observed=False).agg({
    'latitude': 'mean',
    'longitude': 'mean'
})

# Step 6: Merge the city with its lat/lng
city = pd.merge(city, get_lat_lng, on='city_id').reset_index(drop=True)

# Step 7: Correct wrong latitude/longtitude of city (location in Portugal/Spain) to Brazil
city.loc[city['city_name'] == 'porto trombetas', ['latitude','longitude']] = [-1.743514558,-52.24416336]
city.loc[city['city_name'] == 'santa lucia do piai', ['latitude','longitude']]  = [-29.241292800, -51.021271670]
city.loc[city['city_name'] == 'bom retiro da esperanca', ['latitude','longitude']] = [-23.520184363, -48.286817029]
city.loc[city['city_name'] == 'areia branca dos assis', ['latitude','longitude']] = [-25.867626304, -49.368047063]
city.loc[city['city_name'] == 'ilha dos valadares', ['latitude','longitude']] = [-25.533502571, -48.508189284]
#city.loc[city['city_name'] == 'vila nova de campos', ['latitude','longitude']] = [-24.57678608	-53.79553808] # don't know why not working
city.loc[city['city_id'] == 817, ['latitude','longitude']] = [-25.533502571, -48.508189284]

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_city_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'city'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")


/root/.ipykernel/1289/command-2477397622210495-1259402318:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'geolocation_city'] = df['geolocation_city'].str.title()


Cleaned dataset cleaned_olist_geolocation_dataset.csv written successfully!
Data written to geolocation table in Azure PostgreSQL
Cleaned dataset cleaned_olist_city_dataset.csv written successfully!
Data written to city table in Azure PostgreSQL


3. Data Cleaning for "order_items"
- convert columns to appropriate data types
- removed duplicates
- strip leading/trailing whitespace
- handling null values

Issues
- The below code did not work, so changed to allow pandas to automatically infer the format by not specifying the format parameter
    - df['shipping_limit_date'] = pd.to_datetime(df['shipping_limit_date'], format='%d/%m/%Y %I:%M:%S %p')
    - df['shipping_limit_date'] = pd.to_datetime(df['shipping_limit_date'])

In [ ]:
# Load the dataset
file_name = 'olist_order_items_dataset.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))

# Clean the data
def clean_data(df):    
    # Step 1: Remove duplicates in `order_id` + 'order_item_id' if any, keeping the first occurrence
    df = df.drop_duplicates(subset=['order_id','order_item_id'])
    
    # Step 2: Strip leading/trailing whitespace from all string columns
    str_columns = df.select_dtypes(include='string').columns
    df[str_columns] = df[str_columns].apply(lambda col: col.str.strip())

    # Step 3: Replace NULL values with the string "N/A":
    df = df.fillna('N/A')

    # Step 4: Convert columns to appropriate data types
    df['order_id'] = df['order_id'].astype('string')
    df['order_item_id'] = df['order_item_id'].astype('int64')
    df['product_id'] = df['product_id'].astype('string')
    df['seller_id'] = df['seller_id'].astype('string')
    df['shipping_limit_date'] = pd.to_datetime(df['shipping_limit_date'])
    df['price'] = df['price'].apply(Decimal)
    df['freight_value'] = df['freight_value'].apply(Decimal)
    
    # Return the cleaned dataframe
    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_order_items_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'order_items'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")


Cleaned dataset cleaned_olist_order_items_dataset.csv written successfully!
Data written to order_items table in Azure PostgreSQL


4. Data Cleaning for "order_payments"
- removed duplicates
- strip leading/trailing whitespace
- handling null values
- convert columns to appropriate data types

In [ ]:
# Load the dataset
file_name = 'olist_order_payments_dataset.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))

# Clean the data
def clean_data(df):
    # Step 1: Remove duplicates in `order_id`+'pyament_sequential' if any, keeping the first occurrence
    df = df.drop_duplicates(subset=['order_id','payment_sequential'])
    
    # Step 2: Strip leading/trailing whitespace from all string columns
    str_columns = df.select_dtypes(include='string').columns
    df[str_columns] = df[str_columns].apply(lambda col: col.str.strip())

    # Step 3: Increase readibility of the categorical data
    df.loc[:, 'payment_type'] = df['payment_type'].str.replace('_',' ').str.title()

    # Step 4: Replace NULL values with the string "N/A":
    df = df.fillna('N/A')
    
    # Step 5: Convert columns to appropriate data types
    df['order_id'] = df['order_id'].astype('string')
    df['payment_sequential'] = df['payment_sequential'].astype('int8')
    df['payment_type'] = df['payment_type'].astype('category')
    df['payment_installments'] = df['payment_installments'].astype('int8')
    df['payment_value'] = df['payment_value'].apply(Decimal)
    
    # Return the cleaned dataframe
    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_order_payments_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'order_payments'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")


/root/.ipykernel/1289/command-2477397622210499-1323081223:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'payment_type'] = df['payment_type'].str.replace('_',' ').str.title()


Cleaned dataset cleaned_olist_order_payments_dataset.csv written successfully!
Data written to order_payments table in Azure PostgreSQL


5. Data Cleaning for "order_reviews"
- removed duplicates
- strip leading/trailing whitespace
- handling null values
- convert columns to appropriate data types

Issues 
- retained review_score as 'int64' instead oa 'category' in case we would like to do mathematical operations such as applying some statistical analysis

In [ ]:
# Load the dataset
file_name = 'olist_order_reviews_dataset.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))

# Clean the data
def clean_data(df):
    # Step 1: Remove duplicates in `order_id`+'review_id' if any, keeping the first occurrence
    df = df.drop_duplicates(subset=['order_id','review_id'])
    
    # Step 2: Strip leading/trailing whitespace from all string columns
    str_columns = df.select_dtypes(include='string').columns
    df[str_columns] = df[str_columns].apply(lambda col: col.str.strip())

    # Step 3: Replace NULL values with the string "N/A":
    df = df.fillna('N/A')

    # Step 4: Convert columns to appropriate data types
    df['review_id'] = df['review_id'].astype('string')
    df['order_id'] = df['order_id'].astype('string')
    df['review_score'] = df['review_score'].astype('int64')
    df['review_comment_title'] = df['review_comment_title'].astype('string')
    df['review_comment_message'] = df['review_comment_message'].astype('string')
    df['review_creation_date'] = pd.to_datetime(df['review_creation_date'])
    df['review_answer_timestamp'] = pd.to_datetime(df['review_answer_timestamp']).dt.date
    
    # Return the cleaned dataframe
    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_order_reviews_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'order_reviews'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")


Cleaned dataset cleaned_olist_order_reviews_dataset.csv written successfully!
Data written to order_reviews table in Azure PostgreSQL


6. Data Cleaning for "orders"
- convert columns to appropriate data types
- removed duplicates
- strip leading/trailing whitespace
- handling null values

Issues 
- if set to_datetime first, .fillna later is ok, but if .fillna first, to_datetime later is not ok
- however if set to 'category' first, .fillna later is not ok
- hence the solution here is to set as 'string' first, then .fillna, then set as 'category'

In [ ]:
# Load the dataset
file_name = 'olist_orders_dataset.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))

# Clean the data
def clean_data(df):
    # Step 1: Convert columns to appropriate data types
    df['order_id'] = df['order_id'].astype('string')
    df['customer_id'] = df['customer_id'].astype('string')
    df['order_status'] = df['order_status'].astype('string')
    df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
    df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
    df['order_delivered_carrier_date'] = pd.to_datetime(df['order_delivered_carrier_date'])
    df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
    df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

    # Step 2: Remove duplicates in `order_id` if any, keeping the first occurrence
    df = df.drop_duplicates(subset='order_id')
    
    # Step 3: Strip leading/trailing whitespace from all string columns
    str_columns = df.select_dtypes(include='string').columns
    df[str_columns] = df[str_columns].apply(lambda col: col.str.strip())

    # Step 4: Increase readibility of the categorical data
    df['order_status'] = df['order_status'].str.title()

    # Step 5: Replace NULL values with the string "N/A":
    df = df.fillna('N/A') 

    # Step 6: Convert 'order_status' to category
    df['order_status'] = df['order_status'].astype('category')

    # Step 7: Add customer_unique_id column to orders dataset, then drop the customer_id column        
    file_name = 'olist_customers_dataset.csv'
    blob_name = folder + file_name 
    blob_client = container_client.get_blob_client(blob_name) 
    blob_content = blob_client.download_blob().readall() 

    # Use StringIO to read the content into a Pandas DataFrame 
    customers = pd.read_csv(StringIO(blob_content.decode('utf-8')))
    
    df = df.merge(customers[['customer_id', 'customer_unique_id']], left_on='customer_id', right_on='customer_id', how='left')
    df = df.drop(columns=['customer_id'])
    
    # Return the cleaned dataframe
    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_orders_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'orders'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")


Cleaned dataset cleaned_olist_orders_dataset.csv written successfully!
Data written to orders table in Azure PostgreSQL


7. Data Cleaning for "products"
- removed duplicates
- strip leading/trailing whitespace
- handling null values in string columns
- handling NaN or inf in integer columns
- convert columns to appropriate data types

Issues
- issue is that .fillna has to come before .astype('category'), and some columns had null values which cannot be set .astype('int64')
- hence, step 3 & 4 separated into str columns and int columns, for filling with 'N/A' or '0' respectively
- subsequently, step 5 conversion into 'string', 'category' and 'int64' data types

In [ ]:
# Load the dataset
file_name = 'olist_products_dataset.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))

# Clean the data
def clean_data(df): 
    # Step 1: Translate Portugese category names to English
    products_df = df

    file_name = 'product_category_name_translation.csv'
    blob_name = folder + file_name 
    blob_client = container_client.get_blob_client(blob_name) 
    blob_content = blob_client.download_blob().readall() 

    # Use StringIO to read the content into a Pandas DataFrame 
    translation_df = pd.read_csv(StringIO(blob_content.decode('utf-8')))

    # Improve readibility of the text
    translation_df['product_category_name_english'] = translation_df['product_category_name_english'].str.replace('_',' ').str.title()
    
    # Fix typo errors
    replacements = {
    'Costruction Tools Garden': 'Construction Tools Garden',
    'Home Confort': 'Home Comfort',
    'Costruction Tools Tools': 'Construction Tools Tools',
    'Fashio Female Clothing': 'Fashion Female Clothing',
    'Fashion Childrens Clothes': 'Fashion Children Clothes',
    'Dvd': 'DVD',
    'Cd': 'CD'}
    translation_df['product_category_name_english'] = translation_df['product_category_name_english'].replace(replacements, regex=True)
    
    merged_df = products_df.merge(translation_df, on='product_category_name', how='left')
    merged_df.insert(1, 'product_category_name_english_merged', merged_df['product_category_name_english'])
    merged_dropped_df = merged_df.drop(['product_category_name', 'product_category_name_english'], axis=1)  
    df = merged_dropped_df
    
    # Step 2: Remove duplicates in `product_id` if any, keeping the first occurrence
    df = df.drop_duplicates(subset='product_id')
    
    # Step 3: Strip leading/trailing whitespace from all string columns
    str_columns = df.select_dtypes(include='string').columns
    df[str_columns] = df[str_columns].apply(lambda col: col.str.strip())

    # Step 4: Replace NULL values with the string "N/A" in the string columns:
    str_columns = ['product_id', 'product_category_name_english_merged']
    df[str_columns] = df[str_columns].fillna('N/A')

    # Step 5: Replace NaN or inf with 0 in the integer columns:
    int_columns = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                   'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
    df[int_columns] = df[int_columns].replace([np.nan, np.inf, -np.inf], 0)

    # Step 6: Convert columns to appropriate data types
    df['product_id'] = df['product_id'].astype('string')
    df['product_category_name_english_merged'] = df['product_category_name_english_merged'].astype('category')
    df['product_name_lenght'] = df['product_name_lenght'].astype('int64')  
    df['product_description_lenght'] = df['product_description_lenght'].astype('int64')  
    df['product_photos_qty'] = df['product_photos_qty'].astype('int64')  
    df['product_weight_g'] = df['product_weight_g'].astype('int64') 
    df['product_length_cm'] = df['product_length_cm'].astype('int64') 
    df['product_height_cm'] = df['product_height_cm'].astype('int64') 
    df['product_width_cm'] = df['product_width_cm'].astype('int64')

    # Step 7: Rename columns 'product_name_lenght' and 'product_description_lenght' to the correct spelling of 'length'
    df = df.rename(columns={'product_category_name_english_merged':'product_category','product_name_lenght': 'product_name_length', 'product_description_lenght' : 'product_description_length'})
    
    # Return the cleaned dataframe
    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_products_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'products'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")


Cleaned dataset cleaned_olist_products_dataset.csv written successfully!
Data written to products table in Azure PostgreSQL


8. Data Cleaning for "sellers"
- removed duplicates
- strip leading/trailing whitespace
- handling null values
- convert columns to appropriate data types

In [ ]:
# Load the dataset
file_name = 'olist_sellers_dataset.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))

# Clean the data
def clean_data(df):
    # Step 1: Remove duplicates in `product_id` if any, keeping the first occurrence
    df = df.drop_duplicates(subset='seller_id')
    
    # Step 2: Strip leading/trailing whitespace from all string columns
    str_columns = df.select_dtypes(include='string').columns
    df[str_columns] = df[str_columns].apply(lambda col: col.str.strip())

    # Step 3: Increase readibility of the categorical data
    df['seller_city'] = df['seller_city'].str.title()

    # Step 4: Replace NULL values with the string "N/A":
    df = df.fillna('N/A')

    # Step 5: Convert columns to appropriate data types
    df['seller_id'] = df['seller_id'].astype('string')
    df['seller_zip_code_prefix'] = df['seller_zip_code_prefix'].astype('string')
    df['seller_city'] = df['seller_city'].astype('category') 
    df['seller_state'] = df['seller_state'].astype('category')

    # Step 6: Normalize `customer_zip_code_prefix` to ensure all are 5 characters
    df['seller_zip_code_prefix'] = df['seller_zip_code_prefix'].str.zfill(5)
    
    # Return the cleaned dataframe
    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "cleaned_olist_sellers_dataset.csv"

container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'sellers'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")


Cleaned dataset cleaned_olist_sellers_dataset.csv written successfully!
Data written to sellers table in Azure PostgreSQL


In [ ]:
# State names look up table
# Data from other Kaggler dataset

# Load the dataset
file_name = 'misc/states.csv'
blob_name = folder + file_name 
blob_client = container_client.get_blob_client(blob_name) 
blob_content = blob_client.download_blob().readall() 

# Use StringIO to read the content into a Pandas DataFrame 
data = pd.read_csv(StringIO(blob_content.decode('utf-8')))


def clean_data(df):

    df = df[['UF', 'State']]  # Keep state codes and names only
    df = df.rename(columns={'UF': 'code', 'State' : 'state'})  # Rename UF and State to code and state respectively
    
    # Normalise accented strings to normal alphabet strings
    df['state'] = df['state'].apply(lambda x: unidecode(x))
    
    # Copy converted proper name to title case 
    df['state'] = df['state'].str.title()

    return df

# Clean the dataset
cleaned_data = clean_data(data)

# Write cleaned data to Blob Storage cleaned folder
cleaned_file = "states_lookup.csv"
container_client.get_blob_client(f"{folder}{cleaned_folder}{cleaned_file}").upload_blob(cleaned_data.to_csv(index=False), overwrite=True)
print(f"Cleaned dataset {cleaned_file} written successfully!")

# Write cleaned data to PostgresSQL server
table = 'states'
cleaned_data.to_sql(table, engine, if_exists='replace', index=False)
print(f"Data written to {table} table in Azure PostgreSQL")


Cleaned dataset states_lookup.csv written successfully!
Data written to states table in Azure PostgreSQL
